In [1]:
import pandas as pd
import GT_to_csv
import pred_to_CSV
from pred_to_CSV import LABELS2NUM
from utils import *

import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
pred = pd.read_csv("pred_bb.csv").drop(columns="Unnamed: 0")
gt = pd.read_csv("GT_bb.csv").drop(columns="Unnamed: 0")

In [3]:
results = []
for index, row in pred.iterrows():
    pred_bb = BoundingBox(row["filename"],row["label"], row["x"],row["y"],row["w"], row["h"],)
    for_label = []
    for i, r in gt[(gt["label"]==row["label"]) & (gt["filename"]==row["filename"])].iterrows():
        gt_bb = BoundingBox(r["filename"],r["label"], r["x"],r["y"],r["w"], r["h"],)
        for_label.append(box_iou(pred_bb, gt_bb))
    result = [row["filename"], row["label"],row["confidence"], max(for_label, default = 0)]
    results.append(result)
    

In [4]:
df = pd.DataFrame(results, columns = ["filename", "label","confidence", "IOU"])

In [5]:
scores = df.groupby("filename").max().groupby("label").mean().reset_index()

In [6]:
scores["label"] = scores["label"].apply(lambda x: get_key(x, LABELS2NUM))

In [7]:
scores

,label,confidence,IOU
0,Adidas,96.615658,0.813900
1,Apple Inc-,91.139344,0.746895
2,Coca-Cola,97.626866,0.820520
3,Emirates,93.205128,0.747952
4,Hard Rock Cafe,94.469388,0.785356
5,Mercedes-Benz,91.540000,0.792008
6,NFL,92.965116,0.796275
7,Nike,93.030864,0.705703
8,Pepsi,95.804196,0.809263
9,Puma,93.282759,0.764076


In [13]:
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(df[["label"]])
oe_results.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [8]:
X = df[['label', 'confidence']]
X = pd.get_dummies(data=X, drop_first=True)
Y = df["IOU"]

In [9]:
model = LinearRegression().fit(X, Y)

In [10]:
coeff_parameter = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])
coeff_parameter

,Coefficient
label,0.001794
confidence,0.009509
